In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [14]:
price_data = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Dados_bolsa_interpolar.parquet')
price_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.290425,985,ABEV3.SA
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.275703,227,ABEV3.SA
2,2000-01-12,0.481293,0.481293,0.481293,0.481293,0.268352,1137,ABEV3.SA
3,2000-01-13,0.484589,0.484589,0.484589,0.484589,0.270189,606,ABEV3.SA
4,2000-01-14,0.494478,0.494478,0.494478,0.494478,0.275703,6445,ABEV3.SA


In [15]:
price_data['Date'] = pd.to_datetime(price_data['Date'])
price_data.sort_values(by = ['Ticker','Date'], inplace = True)

price_data['change_in_price'] = price_data['Close'].diff()

mask = price_data['Ticker'] != price_data['Ticker'].shift(1)

price_data['change_in_price'] = np.where(mask == True, np.nan, price_data['change_in_price'])

price_data[price_data.isna().any(axis = 1)]


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.290425,985,ABEV3.SA,NaN
17850,2000-12-21,2.815505,2.988535,2.815174,2.960552,1.685388,74224,ITUB4.SA,NaN
23692,2011-05-02,4.705845,4.851989,4.705845,4.808146,4.428368,29936513,MGLU3.SA,NaN
11774,2000-01-03,5.875000,5.875000,5.875000,5.875000,1.278907,35389440000,PETR4.SA,NaN
5746,2000-01-03,3.500000,3.542500,3.500000,3.500000,1.439834,585600,VALE3.SA,NaN


In [16]:
import numpy as np
import pandas as pd

# Função para suavização exponencial
def exponential_smoothing(data, alpha):
    """
    Aplica a suavização exponencial nos dados.
    :param data: Série de dados (array-like ou pandas Series).
    :param alpha: Fator de suavização (0 < alpha < 1).
    :return: Série suavizada.
    """
    smoothed = np.zeros(len(data))
    smoothed[0] = data[0]  # Inicializa com o primeiro valor
    for t in range(1, len(data)):
        smoothed[t] = alpha * data[t] + (1 - alpha) * smoothed[t-1]
    return smoothed

# Função para calcular o target
def calculate_target(data, d):
    """
    Calcula o target usando a fórmula do sinal da diferença de preços.
    :param data: Série de preços de fechamento (array-like ou pandas Series).
    :param d: Período para calcular a diferença futura.
    :return: Série com os valores do target (-1, 0, 1).
    """
    return np.sign(data.shift(-d) - data)

# Função para calcular o target
def calculate_target_verify(data, d):
    """
    Calcula o target usando a fórmula do sinal da diferença de preços.
    :param data: Série de preços de fechamento (array-like ou pandas Series).
    :param d: Período para calcular a diferença futura.
    :return: Série com os valores do target (-1, 0, 1).
    """
    target = np.sign(data.shift(-d) - data)
    target[target == 0] = -1  # Substitui os 0 por -1
    return target


# Lista de alphas para suavização exponencial
alpha_list = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]
d = 10

# Loop para gerar as colunas de preço suavizado para cada alpha
for alpha in alpha_list:
    smoothed_column = f'Smoothed_Close_{alpha}'  # Nome da coluna para o alpha atual
    price_data[smoothed_column] = price_data.groupby('Ticker')['Close'].transform(
        lambda x: exponential_smoothing(x.values, alpha)
    )
    
    # Calculando o target para d = 10 dias (pode ser ajustado conforme necessário)
    target_column = f'Prediction_{alpha:.2f}'  # Nome da coluna de target
    price_data[target_column] = price_data.groupby('Ticker')[smoothed_column].transform(
        lambda x: calculate_target(x, d)
    )


# Aplicando a suavização exponencial
alpha_verify = 1
price_data['Smoothed_Close_1'] = price_data.groupby('Ticker')['Close'].transform(
    lambda x: exponential_smoothing(x.values, alpha_verify)
)

# Calculando o target para d = 10 dias
d = 10
price_data['Verify'] = price_data.groupby('Ticker')['Smoothed_Close_1'].transform(
    lambda x: calculate_target_verify(x, d)
)

price_data

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.290425,985,ABEV3.SA,NaN,0.520882,-1.0,0.520882,-1.0,0.520882,-1.0,0.520882,-1.0,0.520882,-1.0,0.520882,-1.0,0.520882,-1.0,0.520882,-1.0,0.520882,-1.0,0.520882,-1.0
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.275703,227,ABEV3.SA,-0.026404,0.518242,-1.0,0.515601,-1.0,0.512961,-1.0,0.510320,-1.0,0.507680,-1.0,0.505040,-1.0,0.502399,1.0,0.499759,1.0,0.497118,1.0,0.494478,1.0
2,2000-01-12,0.481293,0.481293,0.481293,0.481293,0.268352,1137,ABEV3.SA,-0.013185,0.514547,-1.0,0.508740,-1.0,0.503460,-1.0,0.498709,1.0,0.494486,1.0,0.490792,1.0,0.487625,1.0,0.484986,1.0,0.482876,1.0,0.481293,1.0
3,2000-01-13,0.484589,0.484589,0.484589,0.484589,0.270189,606,ABEV3.SA,0.003296,0.511551,-1.0,0.503909,-1.0,0.497799,1.0,0.493061,1.0,0.489538,1.0,0.487070,1.0,0.485500,1.0,0.484668,1.0,0.484418,1.0,0.484589,1.0
4,2000-01-14,0.494478,0.494478,0.494478,0.494478,0.275703,6445,ABEV3.SA,0.009889,0.509844,-1.0,0.502023,-1.0,0.496803,1.0,0.493628,1.0,0.492008,1.0,0.491515,1.0,0.491785,1.0,0.492516,1.0,0.493472,1.0,0.494478,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11769,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,53.613460,NaN,52.300837,NaN,51.876392,NaN,51.731886,NaN,51.699938,NaN,51.714321,NaN,51.745470,NaN,51.780795,NaN,51.815846,NaN,51.849998,NaN
11770,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,53.512114,NaN,52.360669,NaN,52.093474,NaN,52.079131,NaN,52.149968,NaN,52.245727,NaN,52.343640,NaN,52.436158,NaN,52.521583,NaN,52.599998,NaN
11771,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,53.427902,NaN,52.422535,NaN,52.266431,NaN,52.315478,NaN,52.409983,NaN,52.500290,NaN,52.572091,NaN,52.623230,NaN,52.655157,NaN,52.669998,NaN
11772,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,53.534112,NaN,52.836028,NaN,52.933502,NaN,53.185287,NaN,53.449992,NaN,53.694117,NaN,53.914628,NaN,54.116647,NaN,54.306517,NaN,54.490002,NaN


In [17]:
# Aplicando o deslocamento de 10 linhas dentro de cada grupo de 'Ticker'
price_data['Close_10'] = price_data.groupby('Ticker')['Close'].shift(-d)


In [18]:
# Período do indicador.
n = 14

# Dias de alta e dias de baixa
up_df, down_df = price_data[['Ticker','change_in_price']].copy(), price_data[['Ticker','change_in_price']].copy()

# Salvando a diferença para dias de alta.
up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0

# Salvando a diferença para dias de baixa
down_df.loc['change_in_price'] = down_df.loc[(down_df['change_in_price'] > 0), 'change_in_price'] = 0

# Colocando as diferenças do dia de baixa em termos absolutos
down_df['change_in_price'] = down_df['change_in_price'].abs()


ewma_up = up_df.groupby('Ticker')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())
ewma_down = down_df.groupby('Ticker')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())

relative_strength = ewma_up / ewma_down

# Calculando o indicador
relative_strength_index = 100.0 - (100.0 / (1.0 + relative_strength))

price_data['down_days'] = down_df['change_in_price']
price_data['up_days'] = up_df['change_in_price']
price_data['RSI'] = relative_strength_index

price_data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI
11769,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,53.613460,NaN,52.300837,NaN,51.876392,NaN,51.731886,NaN,51.699938,NaN,51.714321,NaN,51.745470,NaN,51.780795,NaN,51.815846,NaN,51.849998,NaN,NaN,0.000000,0.340000,30.128698
11770,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,53.512114,NaN,52.360669,NaN,52.093474,NaN,52.079131,NaN,52.149968,NaN,52.245727,NaN,52.343640,NaN,52.436158,NaN,52.521583,NaN,52.599998,NaN,NaN,0.000000,0.750000,44.665010
11771,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,53.427902,NaN,52.422535,NaN,52.266431,NaN,52.315478,NaN,52.409983,NaN,52.500290,NaN,52.572091,NaN,52.623230,NaN,52.655157,NaN,52.669998,NaN,NaN,0.000000,0.070000,45.877603
11772,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,53.534112,NaN,52.836028,NaN,52.933502,NaN,53.185287,NaN,53.449992,NaN,53.694117,NaN,53.914628,NaN,54.116647,NaN,54.306517,NaN,54.490002,NaN,NaN,0.000000,1.820004,67.345288
11773,2025-01-20,54.220001,54.490002,53.799999,54.009998,54.009998,3561500,VALE3.SA,-0.480003,53.581701,NaN,53.070822,NaN,53.256451,NaN,53.515172,NaN,53.729995,NaN,53.883646,NaN,53.981387,NaN,54.031328,NaN,54.039650,NaN,54.009998,NaN,NaN,0.480003,0.000000,60.091853


In [19]:


# Iterando sobre cada alpha
for alpha in alpha_list:
    # Nome da coluna Smoothed_Close correspondente ao alpha
    smoothed_col = f'Smoothed_Close_{alpha}'
    
    # Calculando low_14 e high_14 para o alpha atual
    low_14 = price_data[['Ticker', 'Low']].copy()
    high_14 = price_data[['Ticker', 'High']].copy()
    low_14 = low_14.groupby('Ticker')['Low'].transform(lambda x: x.rolling(window=n).min())
    high_14 = high_14.groupby('Ticker')['High'].transform(lambda x: x.rolling(window=n).max())
    
    # Calculando k_percent para o alpha atual
    k_percent = 100 * ((price_data[smoothed_col] - low_14) / (high_14 - low_14))
    
    # Adicionando as colunas ao DataFrame para o alpha atual
    price_data[f'low_14_{alpha:.2f}'] = low_14
    price_data[f'high_14_{alpha:.2f}'] = high_14
    price_data[f'k_percent_{alpha:.2f}'] = k_percent

# Visualizando o DataFrame resultante
price_data.tail()


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI,low_14_0.10,high_14_0.10,k_percent_0.10,low_14_0.20,high_14_0.20,k_percent_0.20,low_14_0.30,high_14_0.30,k_percent_0.30,low_14_0.40,high_14_0.40,k_percent_0.40,low_14_0.50,high_14_0.50,k_percent_0.50,low_14_0.60,high_14_0.60,k_percent_0.60,low_14_0.70,high_14_0.70,k_percent_0.70,low_14_0.80,high_14_0.80,k_percent_0.80,low_14_0.90,high_14_0.90,k_percent_0.90
11769,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,53.613460,NaN,52.300837,NaN,51.876392,NaN,51.731886,NaN,51.699938,NaN,51.714321,NaN,51.745470,NaN,51.780795,NaN,51.815846,NaN,51.849998,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,60.562017,50.869999,55.400002,31.585807,50.869999,55.400002,22.216176,50.869999,55.400002,19.026193,50.869999,55.400002,18.320942,50.869999,55.400002,18.638440,50.869999,55.400002,19.326055,50.869999,55.400002,20.105854,50.869999,55.400002,20.879616
11770,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,53.512114,NaN,52.360669,NaN,52.093474,NaN,52.079131,NaN,52.149968,NaN,52.245727,NaN,52.343640,NaN,52.436158,NaN,52.521583,NaN,52.599998,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,58.324797,50.869999,55.400002,32.906608,50.869999,55.400002,27.008267,50.869999,55.400002,26.691641,50.869999,55.400002,28.255378,50.869999,55.400002,30.369264,50.869999,55.400002,32.530686,50.869999,55.400002,34.573022,50.869999,55.400002,36.458794
11771,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,53.427902,NaN,52.422535,NaN,52.266431,NaN,52.315478,NaN,52.409983,NaN,52.500290,NaN,52.572091,NaN,52.623230,NaN,52.655157,NaN,52.669998,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,56.465823,50.869999,55.400002,34.272299,50.869999,55.400002,30.826305,50.869999,55.400002,31.909009,50.869999,55.400002,33.995219,50.869999,55.400002,35.988741,50.869999,55.400002,37.573748,50.869999,55.400002,38.702652,50.869999,55.400002,39.407433
11772,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,53.534112,NaN,52.836028,NaN,52.933502,NaN,53.185287,NaN,53.449992,NaN,53.694117,NaN,53.914628,NaN,54.116647,NaN,54.306517,NaN,54.490002,NaN,NaN,0.000000,1.820004,67.345288,50.869999,55.189999,61.669291,50.869999,55.189999,45.509942,50.869999,55.189999,47.766288,50.869999,55.189999,53.594644,50.869999,55.189999,59.722076,50.869999,55.189999,65.373107,50.869999,55.189999,70.477538,50.869999,55.189999,75.153904,50.869999,55.189999,79.549039
11773,2025-01-20,54.220001,54.490002,53.799999,54.009998,54.009998,3561500,VALE3.SA,-0.480003,53.581701,NaN,53.070822,NaN,53.256451,NaN,53.515172,NaN,53.729995,NaN,53.883646,NaN,53.981387,NaN,54.031328,NaN,54.039650,NaN,54.009998,NaN,NaN,0.480003,0.000000,60.091853,50.869999,55.189999,62.770880,50.869999,55.189999,50.944989,50.869999,55.189999,55.241955,50.869999,55.189999,61.230857,50.869999,55.189999,66.203626,50.869999,55.189999,69.760348,50.869999,55.189999,72.022885,50.869999,55.189999,73.178922,50.869999,55.189999,73.371562


In [20]:
# Iterando pelos alphas
for alpha in alpha_list:
    # Nome da coluna Smoothed_Close correspondente ao alpha
    smoothed_col = f'Smoothed_Close_{alpha}'
    
    # Calculando low_14 e high_14 para o grupo
    low_14 = price_data[['Ticker', 'Low']].copy()
    high_14 = price_data[['Ticker', 'High']].copy()
    low_14 = low_14.groupby('Ticker')['Low'].transform(lambda x: x.rolling(window=n).min())
    high_14 = high_14.groupby('Ticker')['High'].transform(lambda x: x.rolling(window=n).max())
    
    # Calculando o indicador %R para o alpha atual
    r_percent = ((high_14 - price_data[smoothed_col]) / (high_14 - low_14)) * (-100)
    
    # Adicionando a nova coluna ao DataFrame
    price_data[f'r_percent_{alpha:.2f}'] = r_percent

# Visualizando as últimas linhas do DataFrame com as novas colunas
price_data.tail()


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI,low_14_0.10,high_14_0.10,k_percent_0.10,low_14_0.20,high_14_0.20,k_percent_0.20,low_14_0.30,high_14_0.30,k_percent_0.30,low_14_0.40,high_14_0.40,k_percent_0.40,low_14_0.50,high_14_0.50,k_percent_0.50,low_14_0.60,high_14_0.60,k_percent_0.60,low_14_0.70,high_14_0.70,k_percent_0.70,low_14_0.80,high_14_0.80,k_percent_0.80,low_14_0.90,high_14_0.90,k_percent_0.90,r_percent_0.10,r_percent_0.20,r_percent_0.30,r_percent_0.40,r_percent_0.50,r_percent_0.60,r_percent_0.70,r_percent_0.80,r_percent_0.90
11769,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,53.613460,NaN,52.300837,NaN,51.876392,NaN,51.731886,NaN,51.699938,NaN,51.714321,NaN,51.745470,NaN,51.780795,NaN,51.815846,NaN,51.849998,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,60.562017,50.869999,55.400002,31.585807,50.869999,55.400002,22.216176,50.869999,55.400002,19.026193,50.869999,55.400002,18.320942,50.869999,55.400002,18.638440,50.869999,55.400002,19.326055,50.869999,55.400002,20.105854,50.869999,55.400002,20.879616,-39.437983,-68.414193,-77.783824,-80.973807,-81.679058,-81.361560,-80.673945,-79.894146,-79.120384
11770,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,53.512114,NaN,52.360669,NaN,52.093474,NaN,52.079131,NaN,52.149968,NaN,52.245727,NaN,52.343640,NaN,52.436158,NaN,52.521583,NaN,52.599998,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,58.324797,50.869999,55.400002,32.906608,50.869999,55.400002,27.008267,50.869999,55.400002,26.691641,50.869999,55.400002,28.255378,50.869999,55.400002,30.369264,50.869999,55.400002,32.530686,50.869999,55.400002,34.573022,50.869999,55.400002,36.458794,-41.675203,-67.093392,-72.991733,-73.308359,-71.744622,-69.630736,-67.469314,-65.426978,-63.541206
11771,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,53.427902,NaN,52.422535,NaN,52.266431,NaN,52.315478,NaN,52.409983,NaN,52.500290,NaN,52.572091,NaN,52.623230,NaN,52.655157,NaN,52.669998,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,56.465823,50.869999,55.400002,34.272299,50.869999,55.400002,30.826305,50.869999,55.400002,31.909009,50.869999,55.400002,33.995219,50.869999,55.400002,35.988741,50.869999,55.400002,37.573748,50.869999,55.400002,38.702652,50.869999,55.400002,39.407433,-43.534177,-65.727701,-69.173695,-68.090991,-66.004781,-64.011259,-62.426252,-61.297348,-60.592567
11772,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,53.534112,NaN,52.836028,NaN,52.933502,NaN,53.185287,NaN,53.449992,NaN,53.694117,NaN,53.914628,NaN,54.116647,NaN,54.306517,NaN,54.490002,NaN,NaN,0.000000,1.820004,67.345288,50.869999,55.189999,61.669291,50.869999,55.189999,45.509942,50.869999,55.189999,47.766288,50.869999,55.189999,53.594644,50.869999,55.189999,59.722076,50.869999,55.189999,65.373107,50.869999,55.189999,70.477538,50.869999,55.189999,75.153904,50.869999,55.189999,79.549039,-38.330709,-54.490058,-52.233712,-46.405356,-40.277924,-34.626893,-29.522462,-24.846096,-20.450961
11773,2025-01-20,54.220001,54.490002,53.799999,54.009998,54.009998,3561500,VALE3.SA,-0.480003,53.581701,NaN,53.070822,NaN,53.256451,NaN,53.515172,NaN,53.729995,NaN,53.883646,NaN,53.981387,NaN,54.031328,NaN,54.039650,NaN,54.009998,NaN,NaN,0.480003,0.000000,60.091853,50.869999,55.189999,62.770880,50.869999,55.189999,50.944989,50.869999,55.189999,55.241955,50.869999,55.189999,61.230857,50.869999,55.189999,66.203626,50.869999,55.189999,69.760348,50.869999,55.189999,72.022885,50.869999,55.189999,73.178922,50.869999,55.189999,73

In [21]:
# Iterando pelos alphas
for alpha in alpha_list:
    # Nome da coluna Smoothed_Close correspondente ao alpha
    smoothed_col = f'Smoothed_Close_{alpha}'
    
    # Calculando o MACD usando a coluna suavizada correspondente
    ema_26 = price_data.groupby('Ticker')[smoothed_col].transform(lambda x: x.ewm(span=26).mean())
    ema_12 = price_data.groupby('Ticker')[smoothed_col].transform(lambda x: x.ewm(span=12).mean())
    macd = ema_12 - ema_26
    
    # Calculando a EMA do MACD
    ema_9_macd = macd.ewm(span=9).mean()
    
    # Adicionando as novas colunas ao DataFrame
    price_data[f'MACD_{alpha:.2f}'] = macd
    price_data[f'MACD_EMA_{alpha:.2f}'] = ema_9_macd

# Visualizando as últimas linhas do DataFrame com as novas colunas
price_data.tail()


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI,low_14_0.10,high_14_0.10,k_percent_0.10,low_14_0.20,high_14_0.20,k_percent_0.20,low_14_0.30,high_14_0.30,k_percent_0.30,low_14_0.40,high_14_0.40,k_percent_0.40,low_14_0.50,high_14_0.50,k_percent_0.50,low_14_0.60,high_14_0.60,k_percent_0.60,low_14_0.70,high_14_0.70,k_percent_0.70,low_14_0.80,high_14_0.80,k_percent_0.80,low_14_0.90,high_14_0.90,k_percent_0.90,r_percent_0.10,r_percent_0.20,r_percent_0.30,r_percent_0.40,r_percent_0.50,r_percent_0.60,r_percent_0.70,r_percent_0.80,r_percent_0.90,MACD_0.10,MACD_EMA_0.10,MACD_0.20,MACD_EMA_0.20,MACD_0.30,MACD_EMA_0.30,MACD_0.40,MACD_EMA_0.40,MACD_0.50,MACD_EMA_0.50,MACD_0.60,MACD_EMA_0.60,MACD_0.70,MACD_EMA_0.70,MACD_0.80,MACD_EMA_0.80,MACD_0.90,MACD_EMA_0.90
11769,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,53.613460,NaN,52.300837,NaN,51.876392,NaN,51.731886,NaN,51.699938,NaN,51.714321,NaN,51.745470,NaN,51.780795,NaN,51.815846,NaN,51.849998,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,60.562017,50.869999,55.400002,31.585807,50.869999,55.400002,22.216176,50.869999,55.400002,19.026193,50.869999,55.400002,18.320942,50.869999,55.400002,18.638440,50.869999,55.400002,19.326055,50.869999,55.400002,20.105854,50.869999,55.400002,20.879616,-39.437983,-68.414193,-77.783824,-80.973807,-81.679058,-81.361560,-80.673945,-79.894146,-79.120384,-1.302429,-1.120255,-1.530146,-1.327018,-1.614043,-1.412690,-1.650342,-1.458028,-1.665675,-1.484969,-1.670533,-1.502068,-1.670105,-1.513351,-1.667153,-1.521012,-1.663121,-1.526346
11770,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,53.512114,NaN,52.360669,NaN,52.093474,NaN,52.079131,NaN,52.149968,NaN,52.245727,NaN,52.343640,NaN,52.436158,NaN,52.521583,NaN,52.599998,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,58.324797,50.869999,55.400002,32.906608,50.869999,55.400002,27.008267,50.869999,55.400002,26.691641,50.869999,55.400002,28.255378,50.869999,55.400002,30.369264,50.869999,55.400002,32.530686,50.869999,55.400002,34.573022,50.869999,55.400002,36.458794,-41.675203,-67.093392,-72.991733,-73.308359,-71.744622,-69.630736,-67.469314,-65.426978,-63.541206,-1.326687,-1.161541,-1.533119,-1.368238,-1.593333,-1.448818,-1.608209,-1.488064,-1.605342,-1.509044,-1.595219,-1.520698,-1.582538,-1.527188,-1.569438,-1.530697,-1.556821,-1.532441
11771,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,53.427902,NaN,52.422535,NaN,52.266431,NaN,52.315478,NaN,52.409983,NaN,52.500290,NaN,52.572091,NaN,52.623230,NaN,52.655157,NaN,52.669998,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,56.465823,50.869999,55.400002,34.272299,50.869999,55.400002,30.826305,50.869999,55.400002,31.909009,50.869999,55.400002,33.995219,50.869999,55.400002,35.988741,50.869999,55.400002,37.573748,50.869999,55.400002,38.702652,50.869999,55.400002,39.407433,-43.534177,-65.727701,-69.173695,-68.090991,-66.004781,-64.011259,-62.426252,-61.297348,-60.592567,-1.337291,-1.196691,-1.513041,-1.397199,-1.545152,-1.468085,-1.538018,-1.498055,-1.519037,-1.511042,-1.497726,-1.516104,-1.477673,-1.517285,-1.460072,-1.516572,-1.445140,-1.514981
11772,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,53.534112,NaN,52.836028,NaN,52.933502,NaN,53.185287,NaN,53.449992,NaN,53.694117,NaN,53.914628,NaN,54.116647,NaN,54.306517,NaN,54.490002,NaN,NaN,0.000000,1.820004,67.345288,50.869999,55.189999,61.669291,50.869999,55.189999,45.509942,50.869999,55.189999,47.766288,50.869999,55.189999,53.594644,50.869999,55.189999,5

In [22]:
# Período do indicador
e = 9

# Iterando pelos alphas
for alpha in alpha_list:
    # Nome da coluna Smoothed_Close correspondente ao alpha
    smoothed_col = f'Smoothed_Close_{alpha}'
    
    # Calculando o Price Rate of Change (ROC) usando a coluna suavizada correspondente
    price_rate_col = f'Price_Rate_Of_Change_{alpha:.2f}'
    price_data[price_rate_col] = price_data.groupby('Ticker')[smoothed_col].transform(lambda x: x.pct_change(periods=e))

# Visualizando as últimas linhas do DataFrame com as novas colunas
price_data.tail()


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI,low_14_0.10,high_14_0.10,k_percent_0.10,low_14_0.20,high_14_0.20,k_percent_0.20,low_14_0.30,high_14_0.30,k_percent_0.30,low_14_0.40,high_14_0.40,k_percent_0.40,low_14_0.50,high_14_0.50,k_percent_0.50,low_14_0.60,high_14_0.60,k_percent_0.60,low_14_0.70,high_14_0.70,k_percent_0.70,low_14_0.80,high_14_0.80,k_percent_0.80,low_14_0.90,high_14_0.90,k_percent_0.90,r_percent_0.10,r_percent_0.20,r_percent_0.30,r_percent_0.40,r_percent_0.50,r_percent_0.60,r_percent_0.70,r_percent_0.80,r_percent_0.90,MACD_0.10,MACD_EMA_0.10,MACD_0.20,MACD_EMA_0.20,MACD_0.30,MACD_EMA_0.30,MACD_0.40,MACD_EMA_0.40,MACD_0.50,MACD_EMA_0.50,MACD_0.60,MACD_EMA_0.60,MACD_0.70,MACD_EMA_0.70,MACD_0.80,MACD_EMA_0.80,MACD_0.90,MACD_EMA_0.90,Price_Rate_Of_Change_0.10,Price_Rate_Of_Change_0.20,Price_Rate_Of_Change_0.30,Price_Rate_Of_Change_0.40,Price_Rate_Of_Change_0.50,Price_Rate_Of_Change_0.60,Price_Rate_Of_Change_0.70,Price_Rate_Of_Change_0.80,Price_Rate_Of_Change_0.90
11769,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,53.613460,NaN,52.300837,NaN,51.876392,NaN,51.731886,NaN,51.699938,NaN,51.714321,NaN,51.745470,NaN,51.780795,NaN,51.815846,NaN,51.849998,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,60.562017,50.869999,55.400002,31.585807,50.869999,55.400002,22.216176,50.869999,55.400002,19.026193,50.869999,55.400002,18.320942,50.869999,55.400002,18.638440,50.869999,55.400002,19.326055,50.869999,55.400002,20.105854,50.869999,55.400002,20.879616,-39.437983,-68.414193,-77.783824,-80.973807,-81.679058,-81.361560,-80.673945,-79.894146,-79.120384,-1.302429,-1.120255,-1.530146,-1.327018,-1.614043,-1.412690,-1.650342,-1.458028,-1.665675,-1.484969,-1.670533,-1.502068,-1.670105,-1.513351,-1.667153,-1.521012,-1.663121,-1.526346,-0.045193,-0.052105,-0.054067,-0.054683,-0.054481,-0.053731,-0.052699,-0.051581,-0.050496
11770,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,53.512114,NaN,52.360669,NaN,52.093474,NaN,52.079131,NaN,52.149968,NaN,52.245727,NaN,52.343640,NaN,52.436158,NaN,52.521583,NaN,52.599998,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,58.324797,50.869999,55.400002,32.906608,50.869999,55.400002,27.008267,50.869999,55.400002,26.691641,50.869999,55.400002,28.255378,50.869999,55.400002,30.369264,50.869999,55.400002,32.530686,50.869999,55.400002,34.573022,50.869999,55.400002,36.458794,-41.675203,-67.093392,-72.991733,-73.308359,-71.744622,-69.630736,-67.469314,-65.426978,-63.541206,-1.326687,-1.161541,-1.533119,-1.368238,-1.593333,-1.448818,-1.608209,-1.488064,-1.605342,-1.509044,-1.595219,-1.520698,-1.582538,-1.527188,-1.569438,-1.530697,-1.556821,-1.532441,-0.043760,-0.047825,-0.047025,-0.045026,-0.042495,-0.039783,-0.037132,-0.034670,-0.032434
11771,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,53.427902,NaN,52.422535,NaN,52.266431,NaN,52.315478,NaN,52.409983,NaN,52.500290,NaN,52.572091,NaN,52.623230,NaN,52.655157,NaN,52.669998,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,56.465823,50.869999,55.400002,34.272299,50.869999,55.400002,30.826305,50.869999,55.400002,31.909009,50.869999,55.400002,33.995219,50.869999,55.400002,35.988741,50.869999,55.400002,37.573748,50.869999,55.400002,38.702652,50.869999,55.400002,39.407433,-43.534177,-65.727701,-69.173695,-68.090991,-66.004781,-64.011259,-62.426252,-61.297348,-60.592567,-1.337291,-1.196691,-1.513041,-1.397199,-1.545152,-1.468085,-1.538018,-1.498055,-1.519037,-1.511042,-1.497726,-1.516104,-1.477673,-1.517285,-1.460072,-1.516572,-1.445140,-1.

In [23]:

# Função OBV ajustada para receber uma coluna de preço suavizado específica
def obv(group, smoothed_col):
    Volume = group['Volume']
    change = group[smoothed_col].diff()

    prev_obv = 0
    obv_values = []

    # Calculando o indicador
    for i, j in zip(change, Volume):
        if i > 0:
            current_obv = prev_obv + j
        elif i < 0:
            current_obv = prev_obv - j
        else:
            current_obv = prev_obv

        prev_obv = current_obv
        obv_values.append(current_obv)
    
    # Retornando em pandas series
    return pd.Series(obv_values, index=group.index)

# Iterando sobre os alphas para calcular o OBV para cada preço suavizado
for alpha in alpha_list:
    smoothed_col = f'Smoothed_Close_{alpha}'  # Nome da coluna de preço suavizado
    obv_col = f'OBV_{alpha:.2f}'  # Nome da coluna OBV correspondente

    # Aplicando a função OBV por grupo (Ticker) para o preço suavizado específico
    obv_groups = price_data.groupby('Ticker').apply(lambda group: obv(group, smoothed_col))

    # Adicionando a coluna OBV ao DataFrame
    price_data[obv_col] = obv_groups.reset_index(level=0, drop=True)

# Exibindo as últimas linhas do DataFrame para verificar as colunas OBV criadas
price_data.tail()


<ipython-input-23-b285989f9736>:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  price_data[obv_col] = obv_groups.reset_index(level=0, drop=True)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI,low_14_0.10,high_14_0.10,k_percent_0.10,low_14_0.20,high_14_0.20,k_percent_0.20,low_14_0.30,high_14_0.30,k_percent_0.30,low_14_0.40,high_14_0.40,k_percent_0.40,low_14_0.50,high_14_0.50,k_percent_0.50,low_14_0.60,high_14_0.60,k_percent_0.60,low_14_0.70,high_14_0.70,k_percent_0.70,low_14_0.80,high_14_0.80,k_percent_0.80,low_14_0.90,high_14_0.90,k_percent_0.90,r_percent_0.10,r_percent_0.20,r_percent_0.30,r_percent_0.40,r_percent_0.50,r_percent_0.60,r_percent_0.70,r_percent_0.80,r_percent_0.90,MACD_0.10,MACD_EMA_0.10,MACD_0.20,MACD_EMA_0.20,MACD_0.30,MACD_EMA_0.30,MACD_0.40,MACD_EMA_0.40,MACD_0.50,MACD_EMA_0.50,MACD_0.60,MACD_EMA_0.60,MACD_0.70,MACD_EMA_0.70,MACD_0.80,MACD_EMA_0.80,MACD_0.90,MACD_EMA_0.90,Price_Rate_Of_Change_0.10,Price_Rate_Of_Change_0.20,Price_Rate_Of_Change_0.30,Price_Rate_Of_Change_0.40,Price_Rate_Of_Change_0.50,Price_Rate_Of_Change_0.60,Price_Rate_Of_Change_0.70,Price_Rate_Of_Change_0.80,Price_Rate_Of_Change_0.90,OBV_0.10,OBV_0.20,OBV_0.30,OBV_0.40,OBV_0.50,OBV_0.60,OBV_0.70,OBV_0.80,OBV_0.90
11769,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,53.613460,NaN,52.300837,NaN,51.876392,NaN,51.731886,NaN,51.699938,NaN,51.714321,NaN,51.745470,NaN,51.780795,NaN,51.815846,NaN,51.849998,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,60.562017,50.869999,55.400002,31.585807,50.869999,55.400002,22.216176,50.869999,55.400002,19.026193,50.869999,55.400002,18.320942,50.869999,55.400002,18.638440,50.869999,55.400002,19.326055,50.869999,55.400002,20.105854,50.869999,55.400002,20.879616,-39.437983,-68.414193,-77.783824,-80.973807,-81.679058,-81.361560,-80.673945,-79.894146,-79.120384,-1.302429,-1.120255,-1.530146,-1.327018,-1.614043,-1.412690,-1.650342,-1.458028,-1.665675,-1.484969,-1.670533,-1.502068,-1.670105,-1.513351,-1.667153,-1.521012,-1.663121,-1.526346,-0.045193,-0.052105,-0.054067,-0.054683,-0.054481,-0.053731,-0.052699,-0.051581,-0.050496,2383887448,1978122170,103058362,554669166,482565926,274635038,528256342,-1073784042,-908072738
11770,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,53.512114,NaN,52.360669,NaN,52.093474,NaN,52.079131,NaN,52.149968,NaN,52.245727,NaN,52.343640,NaN,52.436158,NaN,52.521583,NaN,52.599998,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,58.324797,50.869999,55.400002,32.906608,50.869999,55.400002,27.008267,50.869999,55.400002,26.691641,50.869999,55.400002,28.255378,50.869999,55.400002,30.369264,50.869999,55.400002,32.530686,50.869999,55.400002,34.573022,50.869999,55.400002,36.458794,-41.675203,-67.093392,-72.991733,-73.308359,-71.744622,-69.630736,-67.469314,-65.426978,-63.541206,-1.326687,-1.161541,-1.533119,-1.368238,-1.593333,-1.448818,-1.608209,-1.488064,-1.605342,-1.509044,-1.595219,-1.520698,-1.582538,-1.527188,-1.569438,-1.530697,-1.556821,-1.532441,-0.043760,-0.047825,-0.047025,-0.045026,-0.042495,-0.039783,-0.037132,-0.034670,-0.032434,2348749748,2013259870,138196062,589806866,517703626,309772738,563394042,-1038646342,-872935038
11771,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,53.427902,NaN,52.422535,NaN,52.266431,NaN,52.315478,NaN,52.409983,NaN,52.500290,NaN,52.572091,NaN,52.623230,NaN,52.655157,NaN,52.669998,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,56.465823,50.869999,55.400002,34.272299,50.869999,55.400002,30.826305,50.869999,55.400002,31.909009,50.869999,55.400002,33.995219,50.869999,55.400002,35.988741,50.869999,55.400002,37.573748,50.869999,55.400002,38.702652,50.869999,55.400002,39.407433,-

In [24]:
# Any row that has a `NaN` value will be dropped.
price_data = price_data.dropna()

#### Salvando.

In [ ]:
price_data.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)

#### Se quiser rodar para um Ticker.

In [13]:
price_data = price_data[price_data['Ticker'].isin(['ABEV3.SA'])]
price_data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI,low_14_0.10,high_14_0.10,k_percent_0.10,low_14_0.20,high_14_0.20,k_percent_0.20,low_14_0.30,high_14_0.30,k_percent_0.30,low_14_0.40,high_14_0.40,k_percent_0.40,low_14_0.50,high_14_0.50,k_percent_0.50,low_14_0.60,high_14_0.60,k_percent_0.60,low_14_0.70,high_14_0.70,k_percent_0.70,low_14_0.80,high_14_0.80,k_percent_0.80,low_14_0.90,high_14_0.90,k_percent_0.90,r_percent_0.10,r_percent_0.20,r_percent_0.30,r_percent_0.40,r_percent_0.50,r_percent_0.60,r_percent_0.70,r_percent_0.80,r_percent_0.90,MACD_0.10,MACD_EMA_0.10,MACD_0.20,MACD_EMA_0.20,MACD_0.30,MACD_EMA_0.30,MACD_0.40,MACD_EMA_0.40,MACD_0.50,MACD_EMA_0.50,MACD_0.60,MACD_EMA_0.60,MACD_0.70,MACD_EMA_0.70,MACD_0.80,MACD_EMA_0.80,MACD_0.90,MACD_EMA_0.90,Price_Rate_Of_Change_0.10,Price_Rate_Of_Change_0.20,Price_Rate_Of_Change_0.30,Price_Rate_Of_Change_0.40,Price_Rate_Of_Change_0.50,Price_Rate_Of_Change_0.60,Price_Rate_Of_Change_0.70,Price_Rate_Of_Change_0.80,Price_Rate_Of_Change_0.90,OBV_0.10,OBV_0.20,OBV_0.30,OBV_0.40,OBV_0.50,OBV_0.60,OBV_0.70,OBV_0.80,OBV_0.90
5731,2024-12-27,12.16,12.19,11.97,12.07,12.07,37624000,ABEV3.SA,-0.070001,12.831797,-1.0,12.554056,-1.0,12.348159,-1.0,12.231058,-1.0,12.165146,-1.0,12.126536,-1.0,12.103088,-1.0,12.088173,-1.0,12.077865,-1.0,12.07,-1.0,11.21,0.070001,0.0,24.226632,11.97,14.53,33.663951,11.97,14.53,22.814694,11.97,14.53,14.771826,11.97,14.53,10.197552,11.97,14.53,7.622898,11.97,14.53,6.114690,11.97,14.53,5.198754,11.97,14.53,4.616106,11.97,14.53,4.213462,-66.336049,-77.185306,-85.228174,-89.802448,-92.377102,-93.885310,-94.801246,-95.383894,-95.786538,0.043745,0.092703,-0.017453,0.095829,-0.080376,0.070639,-0.123255,0.046028,-0.150932,0.027040,-0.169311,0.012919,-0.182036,0.002297,-0.191183,-0.005871,-0.197982,-0.012295,-0.041303,-0.081110,-0.099313,-0.104035,-0.102534,-0.098464,-0.093457,-0.088261,-0.083249,-2870349193,-2492979247,-2668196973,-1966999623,-1713300605,-1400963487,-1734731475,-1644662135,-1579711273
5732,2024-12-30,12.06,12.16,11.74,11.74,11.74,38907400,ABEV3.SA,-0.330000,12.722617,-1.0,12.391245,-1.0,12.165711,-1.0,12.034634,-1.0,11.952573,-1.0,11.894614,-1.0,11.848926,-1.0,11.809634,-1.0,11.773786,-1.0,11.74,-1.0,11.39,0.330000,0.0,19.331009,11.74,14.50,35.602088,11.74,14.50,23.595839,11.74,14.50,15.424325,11.74,14.50,10.675168,11.74,14.50,7.701931,11.74,14.50,5.601978,11.74,14.50,3.946615,11.74,14.50,2.522991,11.74,14.50,1.224149,-64.397912,-76.404161,-84.575675,-89.324832,-92.298069,-94.398022,-96.053385,-97.477009,-98.775851,0.013040,0.076770,-0.066624,0.063338,-0.135255,0.029460,-0.179276,0.000967,-0.207120,-0.019792,-0.225709,-0.034807,-0.238926,-0.045948,-0.248883,-0.054473,-0.256775,-0.061191,-0.047292,-0.085231,-0.100223,-0.103416,-0.102491,-0.100827,-0.099711,-0.099578,-0.100514,-2909256593,-2531886647,-2707104373,-2005907023,-1752208005,-1439870887,-1773638875,-1683569535,-1618618673
5733,2025-01-02,11.72,11.89,11.49,11.60,11.60,33417900,ABEV3.SA,-0.139999,12.610356,-1.0,12.232996,-1.0,11.995998,-1.0,11.860781,-1.0,11.776287,-1.0,11.717846,-1.0,11.674678,-1.0,11.641927,-1.0,11.617379,-1.0,11.60,-1.0,11.08,0.139999,0.0,17.590948,11.49,14.50,37.221127,11.49,14.50,24.684261,11.49,14.50,16.810569,11.49,14.50,12.318306,11.49,14.50,9.511194,11.49,14.50,7.569641,11.49,14.50,6.135495,11.49,14.50,5.047422,11.49,14.50,4.231867,-62.778873,-75.315739,-83.189431,-87.681694,-90.488806,-92.430359,-93.864505,-94.952578,-95.768133,-0.020121,0.057392,-0.117013,0.027268,-0.190249,-0.014482,-0.234993,-0.046225,-0.262844,-0.068402,-0.281425,-0.084130,-0.294676,-0.095693,-0.304631,-0.104505,-0.312389,-0.1

In [ ]:
price_data = price_data[price_data['Ticker'].isin(['ITUB4.SA'])]
price_data.tail()

In [ ]:
price_data = price_data[price_data['Ticker'].isin(['MGLU3.SA'])]
price_data.tail()

In [102]:
price_data = price_data[price_data['Ticker'].isin(['PETR4.SA'])]
price_data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI,low_14_0.10,high_14_0.10,k_percent_0.10,low_14_0.20,high_14_0.20,k_percent_0.20,low_14_0.30,high_14_0.30,k_percent_0.30,low_14_0.40,high_14_0.40,k_percent_0.40,low_14_0.50,high_14_0.50,k_percent_0.50,low_14_0.60,high_14_0.60,k_percent_0.60,low_14_0.70,high_14_0.70,k_percent_0.70,low_14_0.80,high_14_0.80,k_percent_0.80,low_14_0.90,high_14_0.90,k_percent_0.90,r_percent_0.10,r_percent_0.20,r_percent_0.30,r_percent_0.40,r_percent_0.50,r_percent_0.60,r_percent_0.70,r_percent_0.80,r_percent_0.90,MACD_0.10,MACD_EMA_0.10,MACD_0.20,MACD_EMA_0.20,MACD_0.30,MACD_EMA_0.30,MACD_0.40,MACD_EMA_0.40,MACD_0.50,MACD_EMA_0.50,MACD_0.60,MACD_EMA_0.60,MACD_0.70,MACD_EMA_0.70,MACD_0.80,MACD_EMA_0.80,MACD_0.90,MACD_EMA_0.90,Price_Rate_Of_Change_0.10,Price_Rate_Of_Change_0.20,Price_Rate_Of_Change_0.30,Price_Rate_Of_Change_0.40,Price_Rate_Of_Change_0.50,Price_Rate_Of_Change_0.60,Price_Rate_Of_Change_0.70,Price_Rate_Of_Change_0.80,Price_Rate_Of_Change_0.90,OBV_0.10,OBV_0.20,OBV_0.30,OBV_0.40,OBV_0.50,OBV_0.60,OBV_0.70,OBV_0.80,OBV_0.90
17835,2024-12-27,36.000000,36.000000,35.610001,35.660000,35.660000,24167200,PETR4.SA,-0.110001,37.649875,-1.0,37.001492,-1.0,36.518488,1.0,36.220080,1.0,36.025930,1.0,35.891076,1.0,35.794376,1.0,35.726123,1.0,35.681923,1.0,35.660000,1.0,36.820000,0.110001,0.000000,20.526365,35.599998,40.759998,39.726295,35.599998,40.759998,27.160720,35.599998,40.759998,17.800177,35.599998,40.759998,12.017076,35.599998,40.759998,8.254485,35.599998,40.759998,5.641040,35.599998,40.759998,3.767008,35.599998,40.759998,2.444283,35.599998,40.759998,1.587692,-60.273705,-72.839280,-82.199823,-87.982924,-91.745515,-94.358960,-96.232992,-97.555717,-98.412308,0.199699,0.355567,0.004863,0.308245,-0.149991,0.221659,-0.247798,0.156459,-0.311362,0.110271,-0.355209,0.076877,-0.387169,0.051907,-0.411444,0.032617,-0.430356,0.017298,-0.031530,-0.060793,-0.074687,-0.080939,-0.083218,-0.083132,-0.081442,-0.078543,-0.074657,-279512648955,-150909929051,3100544117,-157200231747,52727347493,160447705765,161501311149,265285062981,183943810781
17836,2024-12-30,35.779999,36.369999,35.770000,36.189999,36.189999,22355600,PETR4.SA,0.529999,37.503888,-1.0,36.839193,1.0,36.419941,1.0,36.208047,1.0,36.107964,1.0,36.070430,1.0,36.071312,1.0,36.097224,1.0,36.139191,1.0,36.189999,1.0,37.290001,0.000000,0.529999,32.377384,35.599998,40.759998,36.897077,35.599998,40.759998,24.015398,35.599998,40.759998,15.890357,35.599998,40.759998,11.783890,35.599998,40.759998,9.844298,35.599998,40.759998,9.116883,35.599998,40.759998,9.133981,35.599998,40.759998,9.636146,35.599998,40.759998,10.449470,-63.102923,-75.984602,-84.109643,-88.216110,-90.155702,-90.883117,-90.866019,-90.363854,-89.550530,0.129634,0.310381,-0.096300,0.227336,-0.255279,0.126271,-0.349058,0.055356,-0.406156,0.006986,-0.442653,-0.027029,-0.466815,-0.051838,-0.483048,-0.070516,-0.493890,-0.084940,-0.033356,-0.058717,-0.067501,-0.068865,-0.066931,-0.063546,-0.059673,-0.055893,-0.052610,-279535004555,-150932284651,3078188517,-157222587347,52749703093,160470061365,161523666749,265307418581,183966166381
17837,2025-01-02,36.419998,37.090000,36.189999,36.770000,36.770000,30046800,PETR4.SA,0.580002,37.430499,-1.0,36.825354,1.0,36.524959,1.0,36.432828,1.0,36.438982,1.0,36.490172,1.0,36.560394,1.0,36.635445,1.0,36.706920,1.0,36.770000,1.0,37.049999,0.000000,0.580002,43.092658,35.599998,40.759998,35.474815,35.599998,40.759998,23.747210,35.599998,40.759998,17.925587,35.599998,40.759998,16.140117,35.599998,40.759998,16.259378,35.599998,40.759998,17.251428,35.599998,40.759998,18.612315,35.599998,40.759998,2

In [25]:
price_data = price_data[price_data['Ticker'].isin(['VALE3.SA'])]
price_data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.10,Smoothed_Close_0.2,Prediction_0.20,Smoothed_Close_0.3,Prediction_0.30,Smoothed_Close_0.4,Prediction_0.40,Smoothed_Close_0.5,Prediction_0.50,Smoothed_Close_0.6,Prediction_0.60,Smoothed_Close_0.7,Prediction_0.70,Smoothed_Close_0.8,Prediction_0.80,Smoothed_Close_0.9,Prediction_0.90,Smoothed_Close_1,Verify,Close_10,down_days,up_days,RSI,low_14_0.10,high_14_0.10,k_percent_0.10,low_14_0.20,high_14_0.20,k_percent_0.20,low_14_0.30,high_14_0.30,k_percent_0.30,low_14_0.40,high_14_0.40,k_percent_0.40,low_14_0.50,high_14_0.50,k_percent_0.50,low_14_0.60,high_14_0.60,k_percent_0.60,low_14_0.70,high_14_0.70,k_percent_0.70,low_14_0.80,high_14_0.80,k_percent_0.80,low_14_0.90,high_14_0.90,k_percent_0.90,r_percent_0.10,r_percent_0.20,r_percent_0.30,r_percent_0.40,r_percent_0.50,r_percent_0.60,r_percent_0.70,r_percent_0.80,r_percent_0.90,MACD_0.10,MACD_EMA_0.10,MACD_0.20,MACD_EMA_0.20,MACD_0.30,MACD_EMA_0.30,MACD_0.40,MACD_EMA_0.40,MACD_0.50,MACD_EMA_0.50,MACD_0.60,MACD_EMA_0.60,MACD_0.70,MACD_EMA_0.70,MACD_0.80,MACD_EMA_0.80,MACD_0.90,MACD_EMA_0.90,Price_Rate_Of_Change_0.10,Price_Rate_Of_Change_0.20,Price_Rate_Of_Change_0.30,Price_Rate_Of_Change_0.40,Price_Rate_Of_Change_0.50,Price_Rate_Of_Change_0.60,Price_Rate_Of_Change_0.70,Price_Rate_Of_Change_0.80,Price_Rate_Of_Change_0.90,OBV_0.10,OBV_0.20,OBV_0.30,OBV_0.40,OBV_0.50,OBV_0.60,OBV_0.70,OBV_0.80,OBV_0.90
11759,2024-12-27,54.730000,55.080002,54.459999,54.740002,54.740002,16635300,VALE3.SA,-0.269997,56.328998,-1.0,55.332221,-1.0,54.966447,-1.0,54.840631,-1.0,54.807772,-1.0,54.801924,-1.0,54.796952,-1.0,54.784814,-1.0,54.765098,-1.0,54.740002,-1.0,51.849998,0.269997,0.0,36.806222,53.330002,60.189999,43.717168,53.330002,60.189999,29.186887,53.330002,60.189999,23.854897,53.330002,60.189999,22.020839,53.330002,60.189999,21.541851,53.330002,60.189999,21.456606,53.330002,60.189999,21.384122,53.330002,60.189999,21.207187,53.330002,60.189999,20.919779,-56.282832,-70.813113,-76.145103,-77.979161,-78.458149,-78.543394,-78.615878,-78.792813,-79.080221,-0.856589,-0.716997,-1.031080,-0.849476,-1.117711,-0.909796,-1.163699,-0.951508,-1.185256,-0.979688,-1.194099,-0.998476,-1.197019,-1.011167,-1.197522,-1.019984,-1.197284,-1.026331,-0.036864,-0.049589,-0.054749,-0.054982,-0.052739,-0.049363,-0.045552,-0.041707,-0.038062,2598477848,2192712570,317648762,726501566,654398326,394799638,586730342,-1015310042,-849598738
11760,2024-12-30,54.900002,55.189999,54.549999,54.549999,54.549999,11250900,VALE3.SA,-0.190002,56.151098,-1.0,55.175777,-1.0,54.841513,-1.0,54.724378,-1.0,54.678886,-1.0,54.650769,-1.0,54.624085,-1.0,54.596962,-1.0,54.571509,-1.0,54.549999,-1.0,52.599998,0.190002,0.0,35.242227,53.330002,60.189999,41.123874,53.330002,60.189999,26.906355,53.330002,60.189999,22.033696,53.330002,60.189999,20.326194,53.330002,60.189999,19.663039,53.330002,60.189999,19.253179,53.330002,60.189999,18.864195,53.330002,60.189999,18.468819,53.330002,60.189999,18.097782,-58.876126,-73.093645,-77.966304,-79.673806,-80.336961,-80.746821,-81.135805,-81.531181,-81.902218,-0.890655,-0.751729,-1.064313,-0.892443,-1.141571,-0.956151,-1.177118,-0.996630,-1.191250,-1.022000,-1.195987,-1.037978,-1.197177,-1.048369,-1.197300,-1.055447,-1.197249,-1.060515,-0.035577,-0.044532,-0.045622,-0.042657,-0.038353,-0.034012,-0.030210,-0.027171,-0.024938,2587226948,2181461670,306397862,715250666,643147426,383548738,575479442,-1026560942,-860849638
11761,2025-01-02,54.709999,55.099998,54.230000,54.250000,54.250000,17623900,VALE3.SA,-0.299999,55.960988,-1.0,54.990622,-1.0,54.664059,-1.0,54.534627,-1.0,54.464443,-1.0,54.410308,-1.0,54.362226,-1.0,54.319392,-1.0,54.282151,-1.0,54.250000,-1.0,52.669998,0.299999,0.0,32.709994,53.330002,59.939999,39.803145,53.330002,59.939999,25.122852,53.330002,59.939999,20.182416,53.330002,59.939999,18.224291,53.330002,59.939999,17.162505,53.330002,59.939999,16.343516,53.330002,59.939999,15.616100,53.330002,59.939999,14.968

#### Se quiser normal só pular etapa anterior.

In [26]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Para cada alpha, criar as variáveis X_Cols e Y_Cols e treinar o modelo
for alpha in alpha_list:
    # Definindo as colunas de X e Y dinamicamente
    X_Cols = price_data[['RSI',
                         f'k_percent_{alpha:.2f}',
                         f'r_percent_{alpha:.2f}',
                         f'Price_Rate_Of_Change_{alpha:.2f}',
                         f'MACD_{alpha:.2f}',
                         f'MACD_EMA_{alpha:.2f}',
                         f'OBV_{alpha:.2f}']]
    
    Y_Cols = price_data[f'Prediction_{alpha:.2f}']

    # Dividir os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, random_state=0)

    # Criar o classificador Random Forest
    rand_frst_clf = RandomForestClassifier(n_estimators=100, oob_score=True, criterion="gini", random_state=0)

    # Ajustar o modelo
    rand_frst_clf.fit(X_train, y_train)

    # Fazer previsões
    y_pred = rand_frst_clf.predict(X_test)

    # Imprimir a acurácia
    accuracy = accuracy_score(y_test, y_pred, normalize=True) * 100.0
    print(f'Alpha: {alpha}')
    print(f'Correct Prediction (%): {accuracy:.5f}')
    print('---')


Alpha: 0.1
Correct Prediction (%): 84.22104
---
Alpha: 0.2
Correct Prediction (%): 75.16644
---
Alpha: 0.3
Correct Prediction (%): 73.30226
---
Alpha: 0.4
Correct Prediction (%): 71.63782
---
Alpha: 0.5
Correct Prediction (%): 67.44341
---
Alpha: 0.6
Correct Prediction (%): 68.10919
---
Alpha: 0.7
Correct Prediction (%): 67.64314
---
Alpha: 0.8
Correct Prediction (%): 68.04261
---
Alpha: 0.9
Correct Prediction (%): 68.37550
---


In [27]:
import warnings
from datetime import timedelta

# Desativa todos os warnings
warnings.filterwarnings("ignore")

# Filtrar últimos 30 dias
last_30_days = price_data[price_data['Date'] >= price_data['Date'].max() - timedelta(days=30)]

# Para cada alpha, selecionar as colunas de X, gerar previsões e comparar com a coluna 'Verify'
for alpha in alpha_list:
    # Selecionar as colunas de X como um array NumPy
    X_Cols = last_30_days[['RSI',
                           f'k_percent_{alpha:.2f}',
                           f'r_percent_{alpha:.2f}',
                           f'Price_Rate_Of_Change_{alpha:.2f}',
                           f'MACD_{alpha:.2f}',
                           f'MACD_EMA_{alpha:.2f}',
                           f'OBV_{alpha:.2f}']].values

    # Gerar previsões e atualizar a coluna 'Prediction_{alpha}' para os últimos 30 dias
    last_30_days[f'Prediction_{alpha:.2f}'] = rand_frst_clf.predict(X_Cols)

    # Comparar as previsões de 'Prediction_{alpha}' com 'Verify' e calcular a precisão
    last_30_days['Match'] = (last_30_days[f'Prediction_{alpha:.2f}'] == last_30_days['Verify']).astype(int)  # 1 para igual, 0 para diferente
    accuracy = last_30_days['Match'].mean()  # Proporção de acertos (iguais / total)

    # Imprimir a precisão para cada alpha
    print(f'\nPrecisão para Alpha: {alpha}')
    print(f'Precisão do modelo nos últimos 30 dias: {accuracy:.2%}')



Precisão para Alpha: 0.1
Precisão do modelo nos últimos 30 dias: 23.53%

Precisão para Alpha: 0.2
Precisão do modelo nos últimos 30 dias: 70.59%

Precisão para Alpha: 0.3
Precisão do modelo nos últimos 30 dias: 47.06%

Precisão para Alpha: 0.4
Precisão do modelo nos últimos 30 dias: 52.94%

Precisão para Alpha: 0.5
Precisão do modelo nos últimos 30 dias: 52.94%

Precisão para Alpha: 0.6
Precisão do modelo nos últimos 30 dias: 64.71%

Precisão para Alpha: 0.7
Precisão do modelo nos últimos 30 dias: 58.82%

Precisão para Alpha: 0.8
Precisão do modelo nos últimos 30 dias: 41.18%

Precisão para Alpha: 0.9
Precisão do modelo nos últimos 30 dias: 94.12%


In [28]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score

# Definir o número de divisões para a validação cruzada
n_splits = 5

# Criar listas para armazenar os resultados gerais
all_predictions = []
all_verify_values = []
all_actual_values = []
all_accuracy_scores_ytest = []
all_accuracy_scores_verify = []

# Loop para cada alpha e realização de validação cruzada em série temporal
for alpha in alpha_list:
    # Definir as features (X), o target (y) e a verificação (verify) dinamicamente
    X = price_data[['RSI',
                    f'k_percent_{alpha:.2f}',
                    f'r_percent_{alpha:.2f}',
                    f'Price_Rate_Of_Change_{alpha:.2f}',
                    f'MACD_{alpha:.2f}',
                    f'MACD_EMA_{alpha:.2f}',
                    f'OBV_{alpha:.2f}']]
    y = price_data[f'Prediction_{alpha:.2f}']
    verify = price_data['Verify']

    # Remover possíveis valores NaN nas colunas selecionadas
    data = pd.concat([X, y, verify], axis=1).dropna()
    X = data.iloc[:, :-2]
    y = data.iloc[:, -2]
    verify = data.iloc[:, -1]

    # Configurar o TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)

    print(f'Alpha: {alpha}')
    fold = 1
    for train_index, test_index in tscv.split(X):
        # Dividir os dados em treino e teste com base nos índices
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        y_verify = verify.iloc[test_index]

        # Criar o modelo de Random Forest
        rand_frst_clf = RandomForestClassifier(n_estimators=100, criterion="gini", random_state=0, oob_score=True)

        # Ajustar o modelo aos dados de treino
        rand_frst_clf.fit(X_train, y_train)

        # Fazer previsões nos dados de teste
        y_pred = rand_frst_clf.predict(X_test)

        # Armazenar as predições e valores reais para análise posterior
        all_predictions.extend(y_pred)
        all_verify_values.extend(y_verify.values)
        all_actual_values.extend(y_test.values)

        # Calcular acurácia para y_test e verify
        accuracy_ytest = accuracy_score(y_test, y_pred) * 100.0
        accuracy_verify = accuracy_score(y_verify, y_pred) * 100.0

        # Armazenar os resultados das acurácias
        all_accuracy_scores_ytest.append(accuracy_ytest)
        all_accuracy_scores_verify.append(accuracy_verify)

        # Exibir resultados por fold
        print(f'Fold {fold}:')
        print(f' - Correct Prediction (y_test): {accuracy_ytest:.2f}%')
        print(f' - Correct Prediction (Verify): {accuracy_verify:.2f}%')
        fold += 1

    print('---')

# Resultados gerais
print("Resumo geral:")
print(f'Média de acurácia (y_test): {np.mean(all_accuracy_scores_ytest):.2f}%')
print(f'Média de acurácia (Verify): {np.mean(all_accuracy_scores_verify):.2f}%')


Alpha: 0.1
Fold 1:
 - Correct Prediction (y_test): 62.50%
 - Correct Prediction (Verify): 52.70%
Fold 2:
 - Correct Prediction (y_test): 71.10%
 - Correct Prediction (Verify): 47.20%
Fold 3:
 - Correct Prediction (y_test): 58.20%
 - Correct Prediction (Verify): 46.90%
Fold 4:
 - Correct Prediction (y_test): 68.40%
 - Correct Prediction (Verify): 49.50%
Fold 5:
 - Correct Prediction (y_test): 72.00%
 - Correct Prediction (Verify): 51.80%
---
Alpha: 0.2
Fold 1:
 - Correct Prediction (y_test): 53.70%
 - Correct Prediction (Verify): 52.00%
Fold 2:
 - Correct Prediction (y_test): 59.80%
 - Correct Prediction (Verify): 49.30%
Fold 3:
 - Correct Prediction (y_test): 57.10%
 - Correct Prediction (Verify): 48.50%
Fold 4:
 - Correct Prediction (y_test): 59.90%
 - Correct Prediction (Verify): 50.20%
Fold 5:
 - Correct Prediction (y_test): 62.70%
 - Correct Prediction (Verify): 51.50%
---
Alpha: 0.3
Fold 1:
 - Correct Prediction (y_test): 39.00%
 - Correct Prediction (Verify): 43.00%
Fold 2:
 - Co

#### Time Series sem Verify

In [30]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score

# Definir o número de divisões para a validação cruzada
n_splits = 5

# Loop para cada alpha e realização de validação cruzada em série temporal
for alpha in alpha_list:
    # Definir as features (X) e o target (Y) dinamicamente
    X = price_data[['RSI',
                    f'k_percent_{alpha:.2f}',
                    f'r_percent_{alpha:.2f}',
                    f'Price_Rate_Of_Change_{alpha:.2f}',
                    f'MACD_{alpha:.2f}',
                    f'MACD_EMA_{alpha:.2f}',
                    f'OBV_{alpha:.2f}']]
    y = price_data[f'Prediction_{alpha:.2f}']

    # Remover possíveis valores NaN nas colunas selecionadas
    data = pd.concat([X, y], axis=1).dropna()
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]

    # Configurar o TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)

    print(f'Alpha: {alpha}')
    fold = 1
    for train_index, test_index in tscv.split(X):
        # Dividir os dados em treino e teste com base nos índices
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Criar o modelo de Random Forest
        rand_frst_clf = RandomForestClassifier(n_estimators=100, criterion="gini", random_state=0,oob_score = True)

        # Ajustar o modelo aos dados de treino
        rand_frst_clf.fit(X_train, y_train)

        # Fazer previsões nos dados de teste
        y_pred = rand_frst_clf.predict(X_test)

        # Calcular e imprimir a acurácia para cada divisão
        accuracy = accuracy_score(y_test, y_pred) * 100.0
        print(f'Fold {fold}: Correct Prediction (%): {accuracy:.2f}')
        fold += 1

    print('---')


Alpha: 0.1
Fold 1: Correct Prediction (%): 62.50
Fold 2: Correct Prediction (%): 71.10
Fold 3: Correct Prediction (%): 58.20
Fold 4: Correct Prediction (%): 68.40
Fold 5: Correct Prediction (%): 72.00
---
Alpha: 0.2
Fold 1: Correct Prediction (%): 53.70
Fold 2: Correct Prediction (%): 59.80
Fold 3: Correct Prediction (%): 57.10
Fold 4: Correct Prediction (%): 59.90
Fold 5: Correct Prediction (%): 62.70
---
Alpha: 0.3
Fold 1: Correct Prediction (%): 39.00
Fold 2: Correct Prediction (%): 54.90
Fold 3: Correct Prediction (%): 53.60
Fold 4: Correct Prediction (%): 54.70
Fold 5: Correct Prediction (%): 56.10
---
Alpha: 0.4
Fold 1: Correct Prediction (%): 41.50
Fold 2: Correct Prediction (%): 49.80
Fold 3: Correct Prediction (%): 48.30
Fold 4: Correct Prediction (%): 49.60
Fold 5: Correct Prediction (%): 57.80
---
Alpha: 0.5
Fold 1: Correct Prediction (%): 49.70
Fold 2: Correct Prediction (%): 48.20
Fold 3: Correct Prediction (%): 46.30
Fold 4: Correct Prediction (%): 48.90
Fold 5: Correct P

In [ ]:
# Salvar o modelo treinado
joblib.dump(rand_frst_clf, 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model.pkl')

In [ ]:
price_data.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)